In [1]:
import theano
import theano.tensor as T
import theano_models as tm
theano.compile.optdb.__position__
from schlichtanders.mylists import remove_duplicates, deflatten

from theano.tensor.elemwise import Elemwise
from theano import scalar
from theano.tensor.shared_randomstreams import RandomStreams

In [2]:
# first step like OpFromGraph
from theano.compile import SharedVariable, rebuild_collect_shared
from theano import gof
from itertools import izip
from theano.printing import debugprint as dp

In [3]:
rng = RandomStreams()

In [4]:
inputs = a, b = T.scalars("ab")
r = rng.random_integers()
r.name = "r"
output = o = a + (2*a + b + r) + (a+(2*a+(b+b+r)))
output.name = "o"
dp(output)

noise_source = tm.list_random_sources(r)
dp(noise_source)

Elemwise{add,no_inplace} [id A] 'o'   
 |Elemwise{add,no_inplace} [id B] ''   
 | |a [id C]
 | |Elemwise{add,no_inplace} [id D] ''   
 |   |Elemwise{add,no_inplace} [id E] ''   
 |   | |Elemwise{mul,no_inplace} [id F] ''   
 |   | | |TensorConstant{2} [id G]
 |   | | |a [id C]
 |   | |b [id H]
 |   |RandomFunction{random_integers_helper}.1 [id I] 'r'   
 |     |<RandomStateType> [id J]
 |     |Shape [id K] ''   
 |     | |Elemwise{add,no_inplace} [id L] ''   
 |     |   |TensorConstant{0} [id M]
 |     |   |TensorConstant{1} [id N]
 |     |TensorConstant{0} [id M]
 |     |TensorConstant{1} [id N]
 |Elemwise{add,no_inplace} [id O] ''   
   |a [id C]
   |Elemwise{add,no_inplace} [id P] ''   
     |Elemwise{mul,no_inplace} [id Q] ''   
     | |TensorConstant{2} [id G]
     | |a [id C]
     |Elemwise{add,no_inplace} [id R] ''   
       |Elemwise{add,no_inplace} [id S] ''   
       | |b [id H]
       | |b [id H]
       |RandomFunction{random_integers_helper}.1 [id I] 'r'   
RandomFunction{r

---

---

## Problem: random numbers get deleted by substitution (in general by graph copy, e.g. done by FunctionGraph).

Solution: replace random variables twice, once for graph simplification and then again for function generation (as this is for numericalize)

### first replace for graph visualization

In [5]:
outputs = [output]

In [6]:
def is_rng(var):
    return hasattr(var, 'rng')

noise_source = tm.list_random_sources(outputs)
ninputs, noutputs, givens = tm.rebuild_graph(noise_source + inputs, [output], tm.graphopt_merge_add_mul)
nnoise, ninputs = deflatten(ninputs, [noise_source, inputs])

In [7]:
dp(ninputs + nnoise + noutputs)

a [id A]
b [id B]
<TensorType(int64, scalar)> [id C]
Elemwise{add,no_inplace} [id D] ''   
 |a [id A]
 |a [id A]
 |Elemwise{mul,no_inplace} [id E] ''   
 | |TensorConstant{2.0} [id F]
 | |a [id A]
 |<TensorType(int64, scalar)> [id C]
 |Elemwise{mul,no_inplace} [id E] ''   
 |b [id B]
 |<TensorType(int64, scalar)> [id C]
 |b [id B]
 |b [id B]


small test how clone would work:

In [8]:
r_noutputs = theano.clone(noutputs, replace=givens)
r_nnoise = [givens[r] if r in givens else r for r in nnoise]

In [10]:
dp(ninputs + r_nnoise + r_noutputs)

a [id A]
b [id B]
RandomFunction{random_integers_helper}.1 [id C] 'r'   
 |<RandomStateType> [id D]
 |Shape [id E] ''   
 | |Elemwise{add,no_inplace} [id F] ''   
 |   |TensorConstant{0} [id G]
 |   |TensorConstant{1} [id H]
 |TensorConstant{0} [id G]
 |TensorConstant{1} [id H]
Elemwise{add,no_inplace} [id I] ''   
 |a [id A]
 |a [id A]
 |Elemwise{mul,no_inplace} [id J] ''   
 | |TensorConstant{2.0} [id K]
 | |a [id A]
 |RandomFunction{random_integers_helper}.1 [id L] 'r'   
 | |<RandomStateType> [id D]
 | |Shape [id M] ''   
 | | |Elemwise{add,no_inplace} [id N] ''   
 | |   |TensorConstant{0} [id G]
 | |   |TensorConstant{1} [id H]
 | |TensorConstant{0} [id G]
 | |TensorConstant{1} [id H]
 |Elemwise{mul,no_inplace} [id J] ''   
 |b [id B]
 |RandomFunction{random_integers_helper}.1 [id L] 'r'   
 |b [id B]
 |b [id B]


As you can see the random variable is not replaced but copied  -  this is useless behaviour. For me it seems like another theano bug

In [ ]:
split = 1
sub, sub2 = tm.independent_subgraphs(ninputs[:split] ,nnoise+ ninputs[split:], noutputs)

In [ ]:
sub, len(sub)

In [ ]:
nsub = tm.independent_subgraphs_extend_add_mul(sub)
nsub

In [ ]:
dp(ninputs[split:] + nsub + noutputs)

### second step just by using theano.function(... givens=...)

Note that the random numbers are executed everywhere where givens was given, even if there is no random number in the subtree. This is another bug of theano.

In [ ]:
f_direct = theano.function(ninputs, noutputs, givens=givens)
f_sub = theano.function(ninputs[:split], nsub, givens=givens)
f_out = theano.function(ninputs[split:] + nsub, noutputs, givens=givens)

In [ ]:
dp(f_out.maker.outputs)

In [ ]:
f_direct(2,5)

In [ ]:
f_out(5, *f_sub(2))

In [ ]:
n_sub = f_sub(2)
n_sub

In [ ]:
f_out(5, *n_sub)

Everything seems to work now.

Mind the final trick was to apply almost all optimizations (which works as merging adds/muls is one of the very last optimization steps, done before merging other elemwise operations)